In [4]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as v_utils
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
from collections import OrderedDict
from skimage import io
import imageio
import os

# 2. data load

In [14]:
trans = transforms.Compose([transforms.CenterCrop(108),
                           transforms.Scale(size=64),
                            transforms.ToTensor()])

trainset = dset.ImageFolder(root='/home/snu/study/GAN/DCGAN/DCGAN-tensorflow/data/',transform=trans)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,shuffle=True, num_workers=2,drop_last=True)

train_data = iter(trainloader)
data_length = trainset.__len__()

Process Process-9:
Process Process-10:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 34, in _worker_loop
    r = index_queue.get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 34, in _worker_loo

# 3.Model

In [15]:
def deconv(ch_in,ch_out,k_size=4,stride=2,pad=1,bn=True,momentum=0.9):
    layers = []
    layers.append(nn.ConvTranspose2d(ch_in,ch_out,k_size,stride,padding=pad))
    if bn:
        layers.append(nn.BatchNorm2d(ch_out,momentum=momentum))
    return nn.Sequential(*layers)

def conv(ch_in, ch_out, k_size, stride, pad=1, bn=True,momentum=0.9):
    layers = []
    layers.append(nn.Conv2d(ch_in,ch_out,k_size,stride,padding=pad))
    if bn:
        layers.append(nn.BatchNorm2d(ch_out,momentum=momentum))
    return nn.Sequential(*layers)
    
def normal_init(m,mean,std):
    if isinstance(m,nn.ConvTranspose2d) or isinstance(m,nn.Conv2d):
        m.weight.data.normal_(mean,std)
        b.bias.data.zero_()
        
def bn_init(m):
    if isinstance(m,nn.BatchNorm2d):
        m.weight.data.fill(1)
        m.bias.data.zero_()

In [54]:
class G(nn.Module):
    def __init__(self,dim_z=100,input_size=64,out_dim=128,colch=3):
        super(G,self).__init__()
        
        self.dim_z = dim_z
        self.input_size = input_size
        self.out_dim = out_dim
        self.colch = colch
        
        self.fc = nn.Linear(dim_z, out_dim*8*(int(input_size/16)**2))
        self.deconv1 = deconv(out_dim*8,out_dim*4,4,)
        self.deconv2 = deconv(out_dim*4,out_dim*2,4)
        self.deconv3 = deconv(out_dim*2,out_dim,4)
        self.deconv4 = deconv(out_dim,colch,4,bn=False,pad=1)
        
        self.weight_init()
        
    def weight_init(self,mean=0.,std=0.02):
        for m in self._modules:
            normal_init(m,mean,std)
            bn_init(m)
        
    def forward(self,z):

            out = self.fc(z)
            out = F.relu(out.view(-1,self.out_dim*8
                                  ,int(self.input_size/16),int(self.input_size/16)))
            out = F.relu(self.deconv1(out))
            out = F.relu(self.deconv2(out))
            out = F.relu(self.deconv3(out))
            out = F.tanh(self.deconv4(out))
            return out
        
class D(nn.Module):
    def __init__(self,input_size=64,conv_dim=128, colch=3,leaky=0.2):
        super(D,self).__init__()
        
        self.input_size = input_size
        self.colch = colch
        self.conv_dim = conv_dim
        self.leaky = leaky
        
        self.conv1 = conv(self.colch,conv_dim,4,2,pad=1,bn=False)
        self.conv2 = conv(conv_dim,conv_dim*2,4,2,pad=1)
        self.conv3 = conv(conv_dim*2,conv_dim*4,4,2,pad=1)
        self.conv4 = conv(conv_dim*4,conv_dim*8,4,2,pad=1)
        self.fc = nn.Linear(conv_dim*8*(int(input_size/16)**2),1)
        self.sigm = nn.Sigmoid()
        
        self.weight_init()
        
    def weight_init(self,mean=0.,std=0.02):
        for m in self._modules:
            normal_init(m,mean,std)
            bn_init(m)
            
    def forward(self,x):
        out = F.leaky_relu(self.conv1(x),self.leaky)
        out = F.leaky_relu(self.conv2(out),self.leaky)
        out = F.leaky_relu(self.conv3(out),self.leaky)
        out = F.leaky_relu(self.conv4(out),self.leaky)
        out = out.view(-1,self.conv_dim*8*(int(self.input_size/16)**2))
        out = self.fc(out)
        out = self.sigm(out).squeeze()
        return out

In [60]:
class DCGAN(object):
    def __init__(self,input_size=64,dim_z=100,colch=3,leaky=0.2,
                end_g_dim=64,start_d_dim=64):
        
        self.input_size= input_size
        self.dim_z = dim_z
        self.colch = colch
        self.leaky = leaky
        self.end_g_dim = end_g_dim
        self.start_d_dim = start_d_dim
        
        self.G = None
        self.D = None
        self.G_optim = None
        self.D_optim = None
        self.G_losses = []
        self.D_losses = []
        self.build_model()
        
        self.fixed_z = Variable(torch.Tensor(64,100).uniform_(-1,1)).cuda()
    def build_model(self):
        self.G = G(self.dim_z,self.input_size,self.end_g_dim,self.colch).cuda()
        self.D = D(self.input_size,self.start_d_dim, self.colch, self.leaky).cuda()
    
    def denorm(self, x):
        """Convert range (-1, 1) to (0, 1)"""
        out = (x + 1) / 2
        return out.clamp(0, 1)

    def train(self, data, batch_size=128,
              epoch=10,lr=0.0002, check_step=100):
            
        batch_length = data.__len__()
        
        self.G_optim = torch.optim.Adam(self.G.parameters(),lr=lr,betas = [0.5,0.999])
        self.D_optim = torch.optim.Adam(self.D.parameters(),lr=lr,betas = [0.5,0.999])
        
        loss_func = nn.BCELoss()
        
        counter = 0
        
        if not os.path.isdir("./dcgan_result"):
            os.mkdir("./dcgan_result")

        for e in range(epoch):
            for i,[batch_x,_] in enumerate(data):

                batch_x = Variable(batch_x).cuda()
                # update D
                z = Variable(torch.Tensor(batch_size,self.dim_z).uniform_(-1,1)).cuda()
                
                self.D_optim.zero_grad()
                
                G_fake = self.denorm(self.G.forward(z))
                D_fake = self.D.forward(G_fake)
                D_real = self.D.forward(batch_x)
                
                D_loss = (torch.sum(loss_func(D_fake,Variable(torch.zeros(batch_size)).cuda()))+
                          torch.sum(loss_func(D_real,Variable(torch.ones(batch_size)).cuda())))

                D_loss.backward(retain_variables=False) # retain_variables=False : 다른 backward를 위해서              
                self.D_optim.step()
                
                # update G
                z = Variable(torch.Tensor(batch_size,self.dim_z).uniform_(-1,1)).cuda()
                
                self.G_optim.zero_grad()
                
                G_fake = self.denorm(self.G.forward(z))
                D_fake = self.D.forward(G_fake)
                
                G_loss = torch.sum(loss_func(D_fake,Variable(torch.ones(batch_size)).cuda()))
                
                G_loss.backward()              
                self.G_optim.step()
                
                counter+=1
                if counter % check_step == 0:
                    
                    self.D_losses.append(D_loss.cpu().data.numpy())
                    self.G_losses.append(G_loss.cpu().data.numpy())
                    
                    print("Epoch [%d/%d], Step [%d/%d], D_loss : %.4f, G_loss : %.4f"%(
                      e,epoch,i,batch_length,D_loss.cpu().data.numpy(),G_loss.cpu().data.numpy()))
                    
                    view = self.denorm(self.G.forward(self.fixed_z))
                    v_utils.save_image(view.data[0:64],"./dcgan_result/gen_{}_{}.png".format(e,i), nrow=8)

In [61]:
fixed_z = Variable(torch.rand(30,100)).cuda()
dcgan = DCGAN()

In [ ]:
dcgan.train(trainloader,epoch=50,check_step=100)

/usr/local/lib/python3.5/dist-packages/torch/autograd/__init__.py:92: UserWarning: retain_variables option is deprecated and will be removed in 0.3. Use retain_graph instead.
  warnings.warn("retain_variables option is deprecated and will be removed in 0.3. "


Epoch [0/100], Step [99/1582], D_loss : 0.5150, G_loss : 2.8774
Epoch [0/100], Step [199/1582], D_loss : 0.8844, G_loss : 2.4530
Epoch [0/100], Step [299/1582], D_loss : 1.0193, G_loss : 2.9846
Epoch [0/100], Step [399/1582], D_loss : 1.3897, G_loss : 1.8557
Epoch [0/100], Step [499/1582], D_loss : 0.9164, G_loss : 2.4483
Epoch [0/100], Step [599/1582], D_loss : 0.6929, G_loss : 2.7136
Epoch [0/100], Step [699/1582], D_loss : 0.6222, G_loss : 3.0657
Epoch [0/100], Step [799/1582], D_loss : 1.0650, G_loss : 2.6618
Epoch [0/100], Step [899/1582], D_loss : 1.1801, G_loss : 1.6856
Epoch [0/100], Step [999/1582], D_loss : 1.0990, G_loss : 4.5262
Epoch [0/100], Step [1099/1582], D_loss : 0.5806, G_loss : 3.7533
Epoch [0/100], Step [1199/1582], D_loss : 0.6882, G_loss : 3.5573
Epoch [0/100], Step [1299/1582], D_loss : 0.9715, G_loss : 1.9512
Epoch [0/100], Step [1399/1582], D_loss : 1.0738, G_loss : 4.2090
Epoch [0/100], Step [1499/1582], D_loss : 0.5945, G_loss : 2.7541
Epoch [1/100], Step [

Epoch [8/100], Step [43/1582], D_loss : 0.9185, G_loss : 9.8999
Epoch [8/100], Step [143/1582], D_loss : 0.1718, G_loss : 4.2329
Epoch [8/100], Step [243/1582], D_loss : 0.2007, G_loss : 2.1357
Epoch [8/100], Step [343/1582], D_loss : 0.8263, G_loss : 2.8504
Epoch [8/100], Step [443/1582], D_loss : 0.1143, G_loss : 3.8977
Epoch [8/100], Step [543/1582], D_loss : 0.1683, G_loss : 4.8332
Epoch [8/100], Step [643/1582], D_loss : 0.6274, G_loss : 5.0931
Epoch [8/100], Step [743/1582], D_loss : 0.1399, G_loss : 3.3666
Epoch [8/100], Step [843/1582], D_loss : 0.2299, G_loss : 3.1330
Epoch [8/100], Step [943/1582], D_loss : 0.0630, G_loss : 4.8439
Epoch [8/100], Step [1043/1582], D_loss : 0.1654, G_loss : 3.6241
Epoch [8/100], Step [1143/1582], D_loss : 0.4267, G_loss : 4.1782
Epoch [8/100], Step [1243/1582], D_loss : 0.1558, G_loss : 3.7102
Epoch [8/100], Step [1343/1582], D_loss : 0.7877, G_loss : 4.9013
Epoch [8/100], Step [1443/1582], D_loss : 0.1616, G_loss : 5.0518
Epoch [8/100], Step [

Epoch [15/100], Step [1469/1582], D_loss : 0.0734, G_loss : 4.6518
Epoch [15/100], Step [1569/1582], D_loss : 0.3964, G_loss : 4.2904
Epoch [16/100], Step [87/1582], D_loss : 0.0405, G_loss : 4.7539
Epoch [16/100], Step [187/1582], D_loss : 0.0664, G_loss : 5.1467
Epoch [16/100], Step [287/1582], D_loss : 0.1674, G_loss : 4.8840
Epoch [16/100], Step [387/1582], D_loss : 0.0580, G_loss : 4.7091
Epoch [16/100], Step [487/1582], D_loss : 0.2944, G_loss : 4.3041
Epoch [16/100], Step [587/1582], D_loss : 0.1724, G_loss : 4.6200
Epoch [16/100], Step [687/1582], D_loss : 0.1567, G_loss : 6.2122
Epoch [16/100], Step [787/1582], D_loss : 0.1795, G_loss : 6.3690
Epoch [16/100], Step [887/1582], D_loss : 0.1432, G_loss : 4.1607
Epoch [16/100], Step [987/1582], D_loss : 0.0923, G_loss : 5.5489
Epoch [16/100], Step [1087/1582], D_loss : 0.2592, G_loss : 3.1958
Epoch [16/100], Step [1187/1582], D_loss : 0.0768, G_loss : 4.8534
Epoch [16/100], Step [1287/1582], D_loss : 0.0806, G_loss : 4.7562
Epoch 

# 애니메이션 만들기

In [8]:
# setting a directory
result_dir = './dcgan_result'

# data load
result_load = io.ImageCollection(result_dir + '/*.png')

images=[]
# image들을 gif파일로 저장
for i, result in enumerate(result_load):
        if i%5 ==0:
            images.append(result)

imageio.mimsave('generation_animation.gif',images,fps=10)
